In [1]:
from occams.nlp import TermOrder
from occams.nlp import process_document
from occams.summarize import SummaryUnits, extract_summary
UNITS = SummaryUnits.CHARS

In [2]:
import pandas as pd
df = pd.read_csv('DUC_TAC_2008_tabular.csv.gz')

In [3]:
# Keeping everything for now, likely will need to filter at some point
df

,sid,mid,n_scus,ann,score_m,score_lq,score_o,summaries,set_a,set_b
0,D0801-A,A,9,A,0.619,5,5,The European Airbus A380 flew its maiden test ...,Airbus mulls boosting A380 production capacity...,NaN
1,D0801-A,C,6,A,0.400,5,4,Emirate Airlines ordered the first passenger A...,Airbus mulls boosting A380 production capacity...,NaN
2,D0801-A,E,9,A,0.636,5,5,"The largest passenger airliner ever built, the...",Airbus mulls boosting A380 production capacity...,NaN
3,D0801-A,G,9,A,0.619,5,5,"In 1994 Airbus began engineering the A380, a s...",Airbus mulls boosting A380 production capacity...,NaN
4,D0801-B,A,8,A,0.818,5,5,Airbus announced it is two to six months behin...,Airbus mulls boosting A380 production capacity...,German wing of Airbus denies superjumbo A380 p...
...,...,...,...,...,...,...,...,...,...,...
379,D0848-A,H,10,H,0.700,5,5,Orhan Pamuk went to court on 16 December for o...,Turkey's Orhan Pamuk stands by remarks about k...,NaN
380,D0848-B,D,4,H,0.529,5,5,Turkey was heavily criticized by international...,Turkey's Orhan Pamuk stands by remarks about k...,TURKISH COURT DROPS CHARGES AGAINST NOVELIST \...
381,D0848-B,E,6,H,0.667,5,5,A group of 169 Turkish intellectuals asked the...,Turkey's Orhan Pamuk stands by remarks about k...,TURKISH COURT DROPS CHARGES AGAINST NOVELIST \...
382,D0848-B,G,4,H,0.529,5,5,Turkey's Justice Ministry declined to rule on ...,Turkey's Orhan Pamuk stands by remarks about k...,TURKISH COURT DROPS CHARGES AGAINST NOVELIST \...


In [4]:
def occams_summarize(document):
     return extract_summary([process_document(document, TermOrder.BIGRAMS)], budget=char_budget, units=UNITS).summary() 

In [11]:
import math
# Determining a size to target for the summary. Adding 50 to give some wiggle room for the length.
avg_len = sum(df['summaries'].str.len())/len(df)
char_budget = math.floor(avg_len) + 50
print(char_budget)

669


In [12]:
# cnn_summaries["occams_summ"] = cnn_summaries.head(10).text.apply(occams_summarize)
from tqdm import tqdm

tqdm.pandas(desc='bar')

df["occams_summ"] = df.set_a.progress_apply(occams_summarize)

bar: 100%|██████████| 384/384 [00:32<00:00, 11.69it/s]


In [15]:
df.to_csv('duc_occams.csv')

In [1]:
import sys
!{sys.executable} -m pip install summac

  Using cached summac-0.0.3-py3-none-any.whl (30 kB)
  Using cached xlrd-1.2.0-py2.py3-none-any.whl (103 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.9 MB/s eta 0:00:00
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached datasets-1.7.0-py3-none-any.whl (234 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20

In [30]:
import pandas as pd
df = pd.read_csv('duc_occams.csv')

In [2]:
# !pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from ipywidgets import FloatProgress


In [1]:
%%time

from summac.model_summac import SummaCZS, SummaCConv

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cpu") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")

document = """Scientists are studying Mars to learn about the Red Planet and find landing sites for future missions.
One possible site, known as Arcadia Planitia, is covered instrange sinuous features.
The shapes could be signs that the area is actually made of glaciers, which are large masses of slow-moving ice.
Arcadia Planitia is in Mars' northern lowlands."""

summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."
score_zs1 = model_zs.score([document], [summary1])
score_conv1 = model_conv.score([document], [summary1])
print("[Summary 1] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

summary2 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers."
score_zs2 = model_zs.score([document], [summary2])
score_conv2 = model_conv.score([document], [summary2])
print("[Summary 2] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs2["scores"][0], score_conv2["scores"][0])) # [Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709

<All keys matched successfully>


/home/vaburban/miniconda3/envs/recommender/lib/python3.8/site-packages/summac/model_summac.py:267: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  histograms = torch.FloatTensor(histograms).to(self.device)


[Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536
[Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709
CPU times: user 1min 10s, sys: 1.53 s, total: 1min 11s
Wall time: 39.1 s


In [3]:
import sys
!{sys.executable} -m pip list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Package                      Version
---------------------------- ----------
absl-py                      1.4.0
aiohttp                      3.8.4
aiosignal                    1.3.1
anyio                        3.7.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
asttokens                    2.2.1
astunparse                   1.6.3
async-timeout                4.0.2
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
bleach                       6.0.0
Bottleneck                   1.3.7
cachetools                   5.3.0
category-encoders            1.3.0


In [4]:
score_zs2 = model_zs.score([document], [document])
score_conv2 = model_conv.score([document], [document])
print(score_zs2)
print(score_conv2)

{'scores': [0.7191451387479901], 'images': [array([[[9.94726717e-01, 2.30021984e-03, 7.93257076e-03, 3.85224372e-02],
        [3.14407703e-03, 9.94307816e-01, 1.05012511e-03, 8.26747157e-03],
        [4.88932803e-03, 8.97570923e-02, 9.93441463e-01, 1.55038005e-02],
        [1.17973762e-03, 6.11782037e-02, 3.36791552e-03, 9.92098212e-01]],

       [[7.47960992e-04, 6.95369765e-03, 1.69338405e-01, 1.30215615e-01],
        [5.89049160e-02, 7.89319980e-04, 3.93543160e-03, 3.13382715e-01],
        [2.27636203e-01, 3.94324139e-02, 6.21550193e-04, 6.47616327e-01],
        [1.16534904e-02, 5.34027182e-02, 3.80042158e-02, 8.12899612e-04]],

       [[4.52523306e-03, 9.90746021e-01, 8.22729051e-01, 8.31261992e-01],
        [9.37951028e-01, 4.90285503e-03, 9.95014489e-01, 6.78349793e-01],
        [7.67474473e-01, 8.70810509e-01, 5.93701378e-03, 3.36879879e-01],
        [9.87166822e-01, 8.85419071e-01, 9.58627820e-01, 7.08891079e-03]]])]}
{'scores': [0.8749397993087769]}


In [2]:
%%time
from summac.model_summac import SummaCZS, SummaCConv

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

document = """Scientists are studying Mars to learn about the Red Planet and find landing sites for future missions.
One possible site, known as Arcadia Planitia, is covered instrange sinuous features.
The shapes could be signs that the area is actually made of glaciers, which are large masses of slow-moving ice.
Arcadia Planitia is in Mars' northern lowlands."""

summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."
score_zs1 = model_zs.score([document], [summary1])
score_conv1 = model_conv.score([document], [summary1])
print("[Summary 1] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

summary2 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers."
score_zs2 = model_zs.score([document], [summary2])
score_conv2 = model_conv.score([document], [summary2])
print("[Summary 2] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs2["scores"][0], score_conv2["scores"][0])) # [Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709

<All keys matched successfully>
[Summary 1] SummaCZS Score: 0.583; SummacConv score: 0.536
[Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709
CPU times: user 1.65 s, sys: 970 ms, total: 2.62 s
Wall time: 5.9 s


In [8]:
# %%time
def runon_df(x):
    model_zs.score(x['occams_summ'], df['set_a'])
    
from tqdm import tqdm

tqdm.pandas(desc='bar')

df['model_zs'] = df.progress_apply(lambda row : model_zs.score([row['set_a']], [row['occams_summ']]), axis = 1)



bar:   0%|          | 0/384 [00:00<?, ?it/s]

AttributeError: 'DataFrame' object has no attribute '_is_builtin_func'

In [23]:
df['occams_summ'].nunique()

48

In [17]:
tiny_df = df.drop_duplicates('occams_summ').head(5)

In [18]:
tiny_df

,Unnamed: 0,sid,mid,n_scus,ann,score_m,score_lq,score_o,summaries,set_a,set_b,occams_summ,model_zs
0,0,D0801-A,A,9,A,0.619,5,5,The European Airbus A380 flew its maiden test ...,Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...,None
8,8,D0802-A,A,8,A,0.579,5,5,Ice continues to melt at an alarming rate in b...,WARMING ANTARCTICA LOOKS EVER MORE VULNERABLE ...,NaN,"But scientists compare the spot where the ""ton...",None
16,16,D0803-A,A,12,A,0.826,5,5,China sacrificed coal mine safety in its massi...,836 officials relinquish stakes in coal mines ...,NaN,"Li Yizhong, director of the National Bureau of...",None
24,24,D0804-A,A,11,A,0.553,5,5,The distress of Hurricane Katrina evacuees for...,Plight of Stranded Animals by Katrina Worsenin...,NaN,Plight of Stranded Animals by Katrina Worsenin...,None
32,32,D0805-A,A,8,F,0.619,5,5,Prescription drug plan Medicare Part D has bee...,SIGN UP BEGINS FOR MEDICARE PRESCRIPTION DRUG ...,NaN,9) The next page lists the cheapest five plans...,None


In [19]:
%%time
from tqdm import tqdm

tqdm.pandas(desc='bar')
tiny_df['model_zs'] = tiny_df.apply(lambda row : model_zs.score([row['set_a']], [row['occams_summ']]), axis = 1)

CPU times: user 55.8 s, sys: 338 ms, total: 56.1 s
Wall time: 54.4 s


In [20]:
tiny_df

,Unnamed: 0,sid,mid,n_scus,ann,score_m,score_lq,score_o,summaries,set_a,set_b,occams_summ,model_zs
0,0,D0801-A,A,9,A,0.619,5,5,The European Airbus A380 flew its maiden test ...,Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...,"{'scores': [-0.1625442420237163], 'images': [[..."
8,8,D0802-A,A,8,A,0.579,5,5,Ice continues to melt at an alarming rate in b...,WARMING ANTARCTICA LOOKS EVER MORE VULNERABLE ...,NaN,"But scientists compare the spot where the ""ton...","{'scores': [-0.3142094211723968], 'images': [[..."
16,16,D0803-A,A,12,A,0.826,5,5,China sacrificed coal mine safety in its massi...,836 officials relinquish stakes in coal mines ...,NaN,"Li Yizhong, director of the National Bureau of...","{'scores': [-0.16270467970106336], 'images': [..."
24,24,D0804-A,A,11,A,0.553,5,5,The distress of Hurricane Katrina evacuees for...,Plight of Stranded Animals by Katrina Worsenin...,NaN,Plight of Stranded Animals by Katrina Worsenin...,"{'scores': [-0.5412645780876891], 'images': [[..."
32,32,D0805-A,A,8,F,0.619,5,5,Prescription drug plan Medicare Part D has bee...,SIGN UP BEGINS FOR MEDICARE PRESCRIPTION DRUG ...,NaN,9) The next page lists the cheapest five plans...,"{'scores': [-0.30491029287313487], 'images': [..."


In [3]:
%%time

from summac.model_summac import SummaCZS, SummaCConv

import pandas as pd
df = pd.read_csv('duc_occams.csv')

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

unique_summs = df.drop_duplicates('occams_summ')
unique_summs['model_zs_oc'] = unique_summs.apply(lambda row : model_zs.score([row['set_a']], [row['occams_summ']])['scores'][0], axis = 1)
unique_summs['model_conv_oc'] = unique_summs.apply(lambda row: model_conv.score([row['set_a']], [row['occams_summ']])['scores'][0], axis = 1)

<All keys matched successfully>


<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/vaburban/miniconda3/envs/recommender/lib/python3.8/site-packages/summac/model_summac.py:267: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  histograms = torch.FloatTensor(histograms).to(self.device)


CPU times: user 5min 24s, sys: 2.74 s, total: 5min 27s
Wall time: 5min 23s


<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
unique_summs['model_zs_oc'].describe()

count    48.000000
mean     -0.075670
std       0.285052
min      -0.666051
25%      -0.229186
50%      -0.101218
75%       0.127014
max       0.564758
Name: model_zs, dtype: float64

In [46]:
unique_summs['model_conv_oc'].describe()

count    48.000000
mean      0.817801
std       0.151037
min       0.463984
25%       0.726551
50%       0.864572
75%       0.938677
max       0.988841
Name: model_conv, dtype: float64

In [14]:
unique_summs = df.drop_duplicates('occams_summ')
unique_summs['model_zs_hum'] = unique_summs.apply(lambda row : model_zs.score([row['set_a']], [row['summaries']])['scores'][0], axis = 1)
unique_summs['model_conv_hum'] = unique_summs.apply(lambda row: model_conv.score([row['set_a']], [row['summaries']])['scores'][0], axis = 1)

/tmp/ipykernel_2388/1162461054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_summs['model_zs_hum'] = unique_summs.apply(lambda row : model_zs.score([row['set_a']], [row['summaries']])['scores'][0], axis = 1)
/tmp/ipykernel_2388/1162461054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_summs['model_conv_hum'] = unique_summs.apply(lambda row: model_conv.score([row['set_a']], [row['summaries']])['scores'][0], axis = 1)


In [15]:
unique_summs['model_zs_hum'].describe()

count    48.000000
mean     -0.448045
std       0.221658
min      -0.880276
25%      -0.612624
50%      -0.411988
75%      -0.331550
max       0.031808
Name: model_zs_hum, dtype: float64

In [16]:
unique_summs['model_conv_hum'].describe()

count    48.000000
mean      0.483429
std       0.122694
min       0.280249
25%       0.380475
50%       0.465457
75%       0.558444
max       0.887520
Name: model_conv_hum, dtype: float64

In [4]:
doc1 = unique_summs['set_a'][112]
summ1 = unique_summs['occams_summ'][112]

print(model_zs.score([doc1], [summ1])['scores'][0])
print(model_conv.score([doc1], [summ1])['scores'][0])

-0.2862701416015625
0.4639836847782135


In [8]:
doc1 = unique_summs['set_a'][112]
summ1 = unique_summs['occams_summ'][112]

print(model_zs.score([doc1.replace('----', ' ')], [summ1])['scores'][0])
print(model_conv.score([doc1.replace('----', ' ')], [summ1])['scores'][0])

-0.2861073811848958
0.4639836847782135


In [58]:
summs = ['A: Senate Majority Leader Bill Frist, a Tennessee Republican, has never said when he would try to change the filibuster rules.', 'Democrats would also clear the way for final votes on William H. Pryor Jr. for the 11th Circuit Court of Appeals and Janice Rogers Brown for the U.S.', 'If the future of filibusters comes to a vote in the Senate, neither side is sure of the outcome.', 'The Senate will then vote on the motion, and a simple majority vote of the 100-member Senate will sustain the ruling of the presiding officer.', 'The Senate would then be poised to vote on the judicial nominee.', 'Owen, a Texas Supreme Court justice nominated to the 5th U.S.']

for summ in summs:
    print(model_zs.score([doc1], [summ])['scores'][0])

-0.7291717529296875
-0.449859619140625
0.1021728515625
-0.3321533203125
-0.25787353515625
-0.0478515625


In [13]:
example_json = {'occams_summ': summ1, 'document': doc1}
import json
with open('duc_summac_example.json', 'w') as f:
    json.dump(example_json, f)

In [ ]:
summac_zs: -0.2861073811848958
summac_conv: 0.4639836847782135

In [60]:
for summ in summs:
    print(model_conv.score([doc1], [summ])['scores'][0])

0.35833147168159485
0.3555246591567993
0.720088541507721
0.4196806848049164
0.4861672818660736
0.42407774925231934


In [63]:
summs[2]

'If the future of filibusters comes to a vote in the Senate, neither side is sure of the outcome.'

In [50]:
unique_summs[unique_summs.model_conv == unique_summs.model_conv.min()]

,Unnamed: 0,sid,mid,n_scus,ann,score_m,score_lq,score_o,summaries,set_a,set_b,occams_summ,model_zs,model_conv
112,112,D0815-A,A,6,C,0.35,5,5,The U.S. Senate remains deadlocked in confirma...,Lawmakers Fail to Reach Filibuster Compromise ...,NaN,"A: Senate Majority Leader Bill Frist, a Tennes...",-0.28627,0.463984


# SMART Eval

In [2]:
from smart_eval import matching_functions as mf
from smart_eval import summeval_utils as utils
from smart_eval import scorer

In [3]:
reference='This article is about this guy named Joe who got arrested.'
candidate='This article is about a guy named Joe'

matcher = mf.chrf_matcher
smart_scorer = scorer.SmartScorer(matching_fn=matcher)
score = smart_scorer.smart_score(reference, candidate)
score

{'smart1': {'recall': 57.804793788108014,
  'precision': 57.804793788108014,
  'fmeasure': 57.804793788108014},
 'smart2': {'recall': 28.902396894054007,
  'precision': 28.902396894054007,
  'fmeasure': 28.902396894054007},
 'smartL': {'recall': 57.804793788108014,
  'precision': 57.804793788108014,
  'fmeasure': 57.804793788108014}}

In [8]:
tiny_df

,Unnamed: 0,sid,mid,n_scus,ann,score_m,score_lq,score_o,summaries,set_a,set_b,occams_summ
0,0,D0801-A,A,9,A,0.619,5,5,The European Airbus A380 flew its maiden test ...,Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...
1,1,D0801-A,C,6,A,0.400,5,4,Emirate Airlines ordered the first passenger A...,Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...


In [4]:
import pandas as pd
df = pd.read_csv('duc_occams.csv')

In [6]:
tiny_df = df.head(2)

In [14]:
matcher = mf.chrf_matcher

def smart_score(reference, candidate):
    smart_scorer = scorer.SmartScorer(matching_fn=matcher)
    score = smart_scorer.smart_score(reference, candidate)
    return {'smart1': score['smart1']['fmeasure'], 'smart2': score['smart2']['fmeasure'], 'smartL': score['smartL']['fmeasure']}

df['model_smarts'] = df.apply(lambda row : smart_score(row['occams_summ'], row['summaries']), axis = 1)

In [16]:
df = pd.concat([df, df["model_smarts"].apply(pd.Series)], axis=1)

In [17]:
df

,Unnamed: 0,sid,mid,n_scus,ann,score_m,score_lq,score_o,summaries,set_a,set_b,occams_summ,model_smarts,smart1,smart2,smartL
0,0,D0801-A,A,9,A,0.619,5,5,The European Airbus A380 flew its maiden test ...,Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...,"{'smart1': 24.77974113427638, 'smart2': 18.797...",24.779741,18.797361,21.836402
1,1,D0801-A,C,6,A,0.400,5,4,Emirate Airlines ordered the first passenger A...,Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...,"{'smart1': 19.227301494637683, 'smart2': 14.92...",19.227301,14.923560,18.706609
2,2,D0801-A,E,9,A,0.636,5,5,"The largest passenger airliner ever built, the...",Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...,"{'smart1': 24.453987278361325, 'smart2': 18.56...",24.453987,18.562996,22.410705
3,3,D0801-A,G,9,A,0.619,5,5,"In 1994 Airbus began engineering the A380, a s...",Airbus mulls boosting A380 production capacity...,NaN,Airbus mulls boosting A380 production capacity...,"{'smart1': 19.300983932571878, 'smart2': 15.45...",19.300984,15.451680,18.550899
4,4,D0801-B,A,8,A,0.818,5,5,Airbus announced it is two to six months behin...,Airbus mulls boosting A380 production capacity...,German wing of Airbus denies superjumbo A380 p...,Airbus mulls boosting A380 production capacity...,"{'smart1': 22.070415104373758, 'smart2': 16.93...",22.070415,16.931291,21.238689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,D0848-A,H,10,H,0.700,5,5,Orhan Pamuk went to court on 16 December for o...,Turkey's Orhan Pamuk stands by remarks about k...,NaN,"""The trial of a novelist who expressed a non-v...","{'smart1': 38.26997122506457, 'smart2': 27.648...",38.269971,27.648868,34.531774
380,380,D0848-B,D,4,H,0.529,5,5,Turkey was heavily criticized by international...,Turkey's Orhan Pamuk stands by remarks about k...,TURKISH COURT DROPS CHARGES AGAINST NOVELIST \...,"""The trial of a novelist who expressed a non-v...","{'smart1': 25.444213160901842, 'smart2': 18.27...",25.444213,18.271141,22.644216
381,381,D0848-B,E,6,H,0.667,5,5,A group of 169 Turkish intellectuals asked the...,Turkey's Orhan Pamuk stands by remarks about k...,TURKISH COURT DROPS CHARGES AGAINST NOVELIST \...,"""The trial of a novelist who expressed a non-v...","{'smart1': 23.779198683360338, 'smart2': 17.65...",23.779199,17.653997,22.739543
382,382,D0848-B,G,4,H,0.529,5,5,Turkey's Justice Ministry declined to rule on ...,Turkey's Orhan Pamuk stands by remarks about k...,TURKISH COURT DROPS CHARGES AGAINST NOVELIST \...,"""The trial of a novelist who expressed a non-v...","{'smart1': 22.51721247141252, 'smart2': 17.350...",22.517212,17.350431,20.824564


In [1]:
!pip list

Package                           Version
--------------------------------- ----------
absl-py                           1.4.0
accelerate                        0.20.1
aiofiles                          22.1.0
aiohttp                           3.8.4
aiosignal                         1.3.1
aiosqlite                         0.19.0
antlr4-python3-runtime            4.9.3
anyio                             3.6.2
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
asttokens                         2.2.1
astunparse                        1.6.3
async-timeout                     4.0.2
attrs                             23.1.0
Babel                             2.12.1
backcall                          0.2.0
beautifulsoup4                    4.12.2
bitsandbytes                      0.39.0
bleach                            6.0.0
blis                              0.7.9
bottle                            0.12.25
cachetools            